In [ ]:
# calculate the RDKit descriptors for MC_DB

In [2]:
import pandas as pd
import numpy as np
from rdkit import DataStructs
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [26]:
df = pd.read_csv('df_modred_new_chems.csv') # load in dataset

In [27]:
df.head()

,chem_category,chem_name,INCHIKEY,OCTANOL_WATER_PARTITION_LOGP_OPERA_PRED,C:M Ratio Master,C:M Ratio Lipid Normalized,Log C:M Ratio Lipid Normalized,C:M Ratio Non-Lipid Normalized,Log C:M Ratio Non-Lipid Normalized,SMILES,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,Amine Co-initiators,"4,4'-Bis(dimethylamino)benzophenone",VVBLNCFGVYUYGU-UHFFFAOYSA-N,3.86903,0.710000,2.257800,0.353685,0.710000,-0.148742,CN(C)C1=CC=C(C=C1)C(=O)C1=CC=C(C=C1)N(C)C,...,9.768984,53.323569,268.157563,6.703939,899,30,100,115,7.777778,4.444444
1,Amine Co-initiators,Ethyl-4-dimethylaminobenzoate,FZUGPQWGEGAKET-UHFFFAOYSA-N,2.98359,0.923077,2.935385,0.467665,0.923077,-0.034762,CCOC(=O)C1=CC=C(C=C1)N(C)C,...,9.182866,44.834790,193.110279,6.658975,331,18,64,71,5.944444,3.305556
2,Amine Co-initiators,2-Ethylhexyl 4-(dimethylamino)benzoate,WYWZRNAHINYAEF-UHFFFAOYSA-N,3.86294,1.062500,3.378750,0.528756,1.062500,0.026329,CCCCC(CC)COC(=O)C1=CC=C(C=C1)N(C)C,...,9.435163,52.355966,277.204179,5.897961,970,26,90,99,8.055556,4.805556
3,Amine Co-initiators,"4,4'-Bis(diethylamino)benzophenone",VYHBFRJRBHMIQZ-UHFFFAOYSA-N,3.93021,11.888889,37.806667,1.577568,11.888889,1.075141,CCN(CC)C1=CC=C(C=C1)C(=O)C1=CC=C(C=C1)N(CC)CC,...,9.913982,58.055175,324.220163,6.235003,1535,38,116,135,8.777778,5.777778
4,Amine Co-initiators,Ethyl-4-aminobenzoate,BLFLLBZGZJTVJG-UHFFFAOYSA-N,1.85996,56.275862,178.957241,2.252749,56.275862,1.750322,CCOC(=O)C1=CC=C(N)C=C1,...,8.952217,41.876660,165.078979,7.177347,211,14,54,59,4.833333,2.861111


In [28]:
df.shape

(264, 1840)

In [29]:
smiles = list(df['SMILES']) # extract the SMILES column and convert it into a list 
smiles

['CN(C)C1=CC=C(C=C1)C(=O)C1=CC=C(C=C1)N(C)C',
 'CCOC(=O)C1=CC=C(C=C1)N(C)C',
 'CCCCC(CC)COC(=O)C1=CC=C(C=C1)N(C)C',
 'CCN(CC)C1=CC=C(C=C1)C(=O)C1=CC=C(C=C1)N(CC)CC',
 'CCOC(=O)C1=CC=C(N)C=C1',
 'O=C(C1=CC=CC=C1)C1=CC=CC=C1',
 'COC(OC)(C(=O)C1=CC=CC=C1)C1=CC=CC=C1',
 'CC1=CC=C(C=C1)C(=O)C1=CC=CC=C1',
 'OC1(CCCCC1)C(=O)C1=CC=CC=C1',
 'CC1=CC=C(C=C1)C(=O)C1=CC=CC=C1',
 'OC1=CC=C(C=C1)C(=O)C1=CC=CC=C1',
 'O=C(C(=O)C1=CC=CC=C1)C1=CC=CC=C1',
 'COC1=CC=C(C(=O)C2=CC=CC=C2)C(O)=C1',
 'CC(C)(C)C1=CC(C=O)=CC(=C1O)C(C)(C)C',
 'CC(C)(C)C1=CC(C)(O)C=C(C1=O)C(C)(C)C',
 'CC(C)(C)C1=CC(=O)C=C(C1=O)C(C)(C)C',
 'CC1=NC=C2CN=C(C3=CC=CC=C3F)C3=CC(Cl)=CC=C3N12',
 'CC(C)OC1=CC=C(C=C1)S(=O)(=O)C1=CC=C(O)C=C1',
 'OC1=CC=C(C=C1)S(=O)(=O)C1=CC=C(O)C=C1',
 'CC(C1=CC=C(O)C=C1)C1=CC=C(O)C=C1',
 'CC(C)(C1=CC=C(O)C=C1)C1=CC=C(O)C=C1',
 'OC1=CC=C(C=C1)C(C1=CC=C(O)C=C1)(C(F)(F)F)C(F)(F)F',
 'COCCCC\\C(=N/OCCN)C1=CC=C(C=C1)C(F)(F)F',
 'COC1=CC=C(C=C1)C(CN(C)C)C1(O)CCCCC1',
 'CC(C)C1=NC(CN(C)C(=O)N[C@@H](CCN2CCOCC2)C(=O)

In [30]:
frags = {} # initiliaze empty dict to store the INCHIKEY and bits 
count = 0 # initiliaze counter 

for i in smiles:
    mol = Chem.MolFromSmiles(i) # convert SMILES to mol 
    fp1 = Chem.RDKFingerprint(mol, maxPath=5,  fpSize = 1024) # calculate RDKit fingerprint
    # show 10 of the set bits:
    list(fp1.GetOnBits())

    fp_arr = np.zeros((1,)) # initiliaze empty numpy array to store bits into
    DataStructs.ConvertToNumpyArray(fp1, fp_arr) # add the bits to the numpy array
    inchi = df.iloc[count,2] # extract the INCHIKEY 
    frags[inchi] = fp_arr # add the INCHIKEY as key and bits as value to the dictionary 
    
    
    count += 1

In [31]:
frags

{'VVBLNCFGVYUYGU-UHFFFAOYSA-N': array([0., 0., 1., ..., 0., 0., 1.]),
 'FZUGPQWGEGAKET-UHFFFAOYSA-N': array([0., 0., 1., ..., 0., 0., 1.]),
 'WYWZRNAHINYAEF-UHFFFAOYSA-N': array([0., 0., 1., ..., 0., 0., 1.]),
 'VYHBFRJRBHMIQZ-UHFFFAOYSA-N': array([0., 0., 1., ..., 0., 0., 1.]),
 'BLFLLBZGZJTVJG-UHFFFAOYSA-N': array([0., 0., 1., ..., 0., 0., 0.]),
 'RWCCWEUUXYIKHB-UHFFFAOYSA-N': array([0., 0., 1., ..., 0., 0., 0.]),
 'KWVGIHKZDCUPEU-UHFFFAOYSA-N': array([0., 1., 1., ..., 0., 0., 0.]),
 'WXPWZZHELZEVPO-UHFFFAOYSA-N': array([0., 0., 1., ..., 0., 0., 0.]),
 'QNODIIQQMGDSEF-UHFFFAOYSA-N': array([1., 0., 1., ..., 0., 0., 1.]),
 'NPFYZDNDJHZQKY-UHFFFAOYSA-N': array([0., 0., 1., ..., 0., 0., 0.]),
 'WURBFLDFSFBTLW-UHFFFAOYSA-N': array([0., 1., 1., ..., 0., 0., 0.]),
 'DXGLGDHPHMLXJC-UHFFFAOYSA-N': array([1., 0., 1., ..., 0., 0., 0.]),
 'DOZRDZLFLOODMB-UHFFFAOYSA-N': array([0., 0., 1., ..., 0., 0., 1.]),
 'DQBHJILNHNRDTM-UHFFFAOYSA-N': array([0., 0., 0., ..., 0., 0., 0.]),
 'RDQSIADLBQFVMY-UHF

In [32]:
len(frags)

260

In [33]:
df_frag = pd.DataFrame.from_dict(frags, orient='index') # convert dict to df
df_frag = df_frag.reset_index()
df_frag.rename(columns = {'index':'INCHIKEY'}, inplace = True)
df_frag

,INCHIKEY,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,VVBLNCFGVYUYGU-UHFFFAOYSA-N,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,FZUGPQWGEGAKET-UHFFFAOYSA-N,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,WYWZRNAHINYAEF-UHFFFAOYSA-N,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,VYHBFRJRBHMIQZ-UHFFFAOYSA-N,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,BLFLLBZGZJTVJG-UHFFFAOYSA-N,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,VLTRZXGMWDSKGL-UHFFFAOYSA-M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
256,WSMYVTOQOOLQHP-UHFFFAOYSA-N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
257,XKJCHHZQLQNZHY-UHFFFAOYSA-N,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
258,ZMZDMBWJUHKJPS-UHFFFAOYSA-M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
df_merged = df.merge(df_frag, on = 'INCHIKEY') # merge the new df with bits to the original df based on INCHIKEY 
df_merged

,chem_category,chem_name,INCHIKEY,OCTANOL_WATER_PARTITION_LOGP_OPERA_PRED,C:M Ratio Master,C:M Ratio Lipid Normalized,Log C:M Ratio Lipid Normalized,C:M Ratio Non-Lipid Normalized,Log C:M Ratio Non-Lipid Normalized,SMILES,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,Amine Co-initiators,"4,4'-Bis(dimethylamino)benzophenone",VVBLNCFGVYUYGU-UHFFFAOYSA-N,3.869030,0.710000,2.257800,0.353685,0.710000,-0.148742,CN(C)C1=CC=C(C=C1)C(=O)C1=CC=C(C=C1)N(C)C,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Amine Co-initiators,Ethyl-4-dimethylaminobenzoate,FZUGPQWGEGAKET-UHFFFAOYSA-N,2.983590,0.923077,2.935385,0.467665,0.923077,-0.034762,CCOC(=O)C1=CC=C(C=C1)N(C)C,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Amine Co-initiators,2-Ethylhexyl 4-(dimethylamino)benzoate,WYWZRNAHINYAEF-UHFFFAOYSA-N,3.862940,1.062500,3.378750,0.528756,1.062500,0.026329,CCCCC(CC)COC(=O)C1=CC=C(C=C1)N(C)C,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Amine Co-initiators,"4,4'-Bis(diethylamino)benzophenone",VYHBFRJRBHMIQZ-UHFFFAOYSA-N,3.930210,11.888889,37.806667,1.577568,11.888889,1.075141,CCN(CC)C1=CC=C(C=C1)C(=O)C1=CC=C(C=C1)N(CC)CC,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,Amine Co-initiators,Ethyl-4-aminobenzoate,BLFLLBZGZJTVJG-UHFFFAOYSA-N,1.859960,56.275862,178.957241,2.252749,56.275862,1.750322,CCOC(=O)C1=CC=C(N)C=C1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Other,perchlorate,VLTRZXGMWDSKGL-UHFFFAOYSA-M,NaN,0.623318,1.982152,0.297137,0.623318,-0.205290,[O-][Cl](=O)(=O)=O,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
260,Other,malondialdehyde,WSMYVTOQOOLQHP-UHFFFAOYSA-N,-0.596378,0.788030,2.505935,0.398970,0.788030,-0.103457,O=CCC=O,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
261,Other,Phthalimide,XKJCHHZQLQNZHY-UHFFFAOYSA-N,1.149400,0.812831,2.584801,0.412427,0.812831,-0.090000,O=C1NC(=O)C2=C1C=CC=C2,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
262,Other,thiocyanate,ZMZDMBWJUHKJPS-UHFFFAOYSA-M,0.560862,0.897436,2.853846,0.455431,0.897436,-0.046997,[S-]C#N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# testing to see if the merge was successful 

In [35]:
df_merged.loc[100, "INCHIKEY"] 

'VAGXZGJKNUNLHK-WJDWOHSUSA-N'

In [36]:
df_merged.loc[100, 'SMILES'] 

'CNCCC=C1C2=CC=CC=C2CC(C3=CC=CC=C31)O'

In [37]:
df_merged.columns = df_merged.columns.astype(str)

In [3]:
mol1 = Chem.MolFromSmiles('CNCCC=C1C2=CC=CC=C2CC(C3=CC=CC=C31)O') 
mol1

rdkfp = Chem.RDKFingerprint(mol1, maxPath=5,  fpSize = 1024)
# show 10 of the set bits:
list(rdkfp.GetOnBits())

[1,
 2,
 4,
 18,
 19,
 22,
 23,
 27,
 28,
 31,
 36,
 40,
 41,
 51,
 52,
 71,
 81,
 91,
 93,
 96,
 103,
 105,
 112,
 114,
 120,
 124,
 134,
 136,
 142,
 148,
 151,
 159,
 161,
 163,
 166,
 170,
 183,
 194,
 200,
 202,
 205,
 209,
 216,
 222,
 226,
 227,
 237,
 238,
 247,
 251,
 254,
 256,
 258,
 276,
 277,
 279,
 284,
 285,
 294,
 301,
 308,
 309,
 313,
 314,
 315,
 319,
 320,
 324,
 329,
 330,
 337,
 339,
 342,
 346,
 356,
 360,
 364,
 368,
 383,
 390,
 395,
 405,
 407,
 412,
 418,
 420,
 421,
 424,
 432,
 433,
 441,
 445,
 446,
 464,
 473,
 480,
 488,
 500,
 501,
 508,
 518,
 519,
 521,
 524,
 525,
 527,
 529,
 544,
 553,
 557,
 562,
 575,
 576,
 579,
 587,
 588,
 590,
 600,
 614,
 625,
 635,
 638,
 646,
 653,
 661,
 670,
 673,
 676,
 677,
 684,
 696,
 701,
 704,
 706,
 708,
 709,
 717,
 726,
 728,
 731,
 744,
 748,
 750,
 756,
 760,
 762,
 779,
 781,
 785,
 789,
 792,
 798,
 805,
 809,
 817,
 818,
 820,
 824,
 826,
 827,
 830,
 831,
 837,
 838,
 842,
 845,
 846,
 852,
 854,
 855,
 85

In [39]:
df_merged.loc[100,"930"]

1.0

In [40]:
array1 = np.array(df_merged.loc[100,'0':'1023'])
array1

array([0.0, 1.0, 1.0, ..., 0.0, 0.0, 1.0], dtype=object)

In [41]:
array1.nonzero()

(array([   1,    2,    4,   18,   19,   22,   23,   27,   28,   31,   36,
          40,   41,   51,   52,   71,   81,   91,   93,   96,  103,  105,
         112,  114,  120,  124,  134,  136,  142,  148,  151,  159,  161,
         163,  166,  170,  183,  194,  200,  202,  205,  209,  216,  222,
         226,  227,  237,  238,  247,  251,  254,  256,  258,  276,  277,
         279,  284,  285,  294,  301,  308,  309,  313,  314,  315,  319,
         320,  324,  329,  330,  337,  339,  342,  346,  356,  360,  364,
         368,  383,  390,  395,  405,  407,  412,  418,  420,  421,  424,
         432,  433,  441,  445,  446,  464,  473,  480,  488,  500,  501,
         508,  518,  519,  521,  524,  525,  527,  529,  544,  553,  557,
         562,  575,  576,  579,  587,  588,  590,  600,  614,  625,  635,
         638,  646,  653,  661,  670,  673,  676,  677,  684,  696,  701,
         704,  706,  708,  709,  717,  726,  728,  731,  744,  748,  750,
         756,  760,  762,  779,  781, 

In [42]:
np.nonzero(frags['VAGXZGJKNUNLHK-WJDWOHSUSA-N']) 
# comparing the bits array from above to the one we get from entering the INCHIKEY into the dict
# and calculating the RDKit manually by entering the SMILES, shows they match

(array([   1,    2,    4,   18,   19,   22,   23,   27,   28,   31,   36,
          40,   41,   51,   52,   71,   81,   91,   93,   96,  103,  105,
         112,  114,  120,  124,  134,  136,  142,  148,  151,  159,  161,
         163,  166,  170,  183,  194,  200,  202,  205,  209,  216,  222,
         226,  227,  237,  238,  247,  251,  254,  256,  258,  276,  277,
         279,  284,  285,  294,  301,  308,  309,  313,  314,  315,  319,
         320,  324,  329,  330,  337,  339,  342,  346,  356,  360,  364,
         368,  383,  390,  395,  405,  407,  412,  418,  420,  421,  424,
         432,  433,  441,  445,  446,  464,  473,  480,  488,  500,  501,
         508,  518,  519,  521,  524,  525,  527,  529,  544,  553,  557,
         562,  575,  576,  579,  587,  588,  590,  600,  614,  625,  635,
         638,  646,  653,  661,  670,  673,  676,  677,  684,  696,  701,
         704,  706,  708,  709,  717,  726,  728,  731,  744,  748,  750,
         756,  760,  762,  779,  781, 

In [43]:
df_merged.to_csv('Fingerprints_RDKit_MC_DB_maxPath5.csv')